# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import cv2
import random
import tensorflow as tf
import seaborn as sns
import os
import random
import keras

from keras import models
from keras import layers
from keras import optimizers
from keras.models import Model, Sequential
from keras.layers import Conv2D, Conv2DTranspose, Reshape
from keras.applications.inception_resnet_v2 import InceptionResNetV2, decode_predictions, preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
from keras.layers import MaxPooling2D, Flatten, Dense, Dropout, Activation, InputLayer, BatchNormalization  
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
                    
import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from matplotlib.image import imread
from sklearn.cluster import KMeans

keras.utils.vis_utils.pydot = pyd

# Import Datasets

In [ ]:
L_images = np.load('l/gray_scale.npy')

In [ ]:
AB_images_1 = np.load('ab/ab/ab1.npy')
AB_images_2 = np.load('ab/ab/ab2.npy')
AB_images_3 = np.load('ab/ab/ab3.npy')

# Exploratory Data Analysis

In [ ]:
L_images.shape

In [ ]:
AB_images_1.shape

In [ ]:
AB_images = np.concatenate((AB_images_1, AB_images_2, AB_images_3))

In [ ]:
AB_images.shape

# Normalizing Data

In [ ]:
# def normalize_L(data):
#     L = []
#     i = 0
#     while i <= 30:
#         for img in np.nditer(data[i]):
#             img /= 100
#         L.append(img)
#         i += 1
#     L = np.array(L, dtype='float')
#     return L

# def normalize_ab(data):
#     for img in np.nditer(data):
#         img = img*(1.0/128)
#     return data

In [ ]:
# L_norm = normalize_L(L_images)

In [ ]:
# AB_norm_1 = normalize_ab(AB_images_1)

# Preprocessing L_images to make RGB

In [ ]:
def prep_L(L_images, batch_size = 300):
    # Create and empty array of given batch size, 224x224, with three channels
    # so that the L channel can fall in a tensor that will eventually hold all three
    # RGB channels
    L_imgs = np.zeros((batch_size, 224, 224, 3))
    # fFr every channel in range 2
    for i in range(0, 3):
        # a new image in the RGB_imgs array will be a greyscale image from L_images
        L_imgs[:batch_size, :, :,i] = L_images[:batch_size]
    # Return the standardized version of this new array.
    return preprocess_input(L_imgs)

# Creating RGB targets from L and AB values

In [ ]:
def make_RGB(L_images, AB_images, batch_size = 300):
    # Create and empty array of the appropriate size to hold
    # L and AB images. It will be of batch size, 224x244
    # and be composed of three color channels for RGB
    RGB_imgs = np.zeros((batch_size, 224, 224, 3))
    # The first color channel will be the L in L_images, up to the batch size
    RGB_imgs[:, :, :, 0] = L_images[0:batch_size]
    # The second /third color channels will be the AB from AB_images
    RGB_imgs[:, :, :, 1:] = AB_images[0:batch_size]
    # Convert everything to the same file type
    RGB_imgs = RGB_imgs.astype("uint8")
    # Create a new empty list to hold all of the RGB images
    RGB_array = []
    # for every image in the range of the batch size, convert LAB to RGB
    # and then append to our empty list of imgs_
    for i in range(0, batch_size):
        RGB_array.append(cv2.cvtColor(RGB_imgs[i], cv2.COLOR_LAB2RGB))
    # Turn our new list of RGB images back into an array   
    RGB_array = np.array(RGB_array)
    return RGB_array

# Creating Train and Target Data

In [ ]:
imgs_input = prep_L(L_images, batch_size = 300)

In [ ]:
imgs_output = preprocess_input(make_RGB(L_images, AB_images, batch_size = 300))

In [ ]:
imgs_input.dtype

In [ ]:
plt.imshow(L_images[25], cmap='gray')

In [ ]:
img = imgs_output[25]
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
L, A, B = cv2.split(lab)
plt.imshow(A)

In [ ]:
plt.imshow(B)

In [ ]:
plt.imshow(imgs_output[25]);

# Building a Baseline Convolutional Neural Network

In [ ]:
model_simple = Sequential()
model_simple.add(Conv2D(strides = 1, kernel_size = 3, filters = 12, padding = "same", activation = 'relu'))
model_simple.add(Conv2D(strides = 1, kernel_size = 3, filters = 12, padding = "same", activation = 'relu'))
model_simple.add(Conv2DTranspose(strides = 1, kernel_size = 3, filters = 12, padding = "same", activation = 'relu'))
model_simple.add(Conv2DTranspose(strides = 1, kernel_size = 3, filters = 3, padding = "same", activation = 'relu'))

In [ ]:
model_simple.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [ ]:
model_simple.fit(imgs_for_input, imgs_for_output, epochs = 15)

In [ ]:
print(model_simple.evaluate(imgs_for_input, imgs_for_output))

# Predicting Baseline Output

In [ ]:
output = model_simple.predict(imgs_for_input)

In [ ]:
plt.imshow(output[30])

In [ ]:
plt.imshow(imgs_for_output[30])

# Evaluating Baseline Performance

In [ ]:
h = model_simple.history

plt.plot(h.history['acc'])
plt.title('Model accuracy')
plt.show()

plt.plot(h.history['loss'])
plt.title('Model Loss')
plt.show()

 # Creating a more powerful CNN with greater Feature Maps and Upsampling

In [ ]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(224,224,3)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))

In [ ]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse', metrics=['acc'])

In [ ]:
model.fit(imgs_for_input, imgs_for_output, epochs = 15)

# Predicting updated CNN Outputs

In [ ]:
output = model.predict(imgs_for_input)

In [ ]:
plt.imshow(output[4])

In [ ]:
plt.imshow(imgs_for_output[4])

# Building a Fully Connected Network with no Upsampling or Pooling

In [ ]:
# Building the neural network
updated_model = Sequential()
updated_model.add(InputLayer(input_shape=(224,224,3)))
updated_model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=1))
updated_model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
updated_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
updated_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=1))
updated_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
updated_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=1))
updated_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
updated_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
updated_model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))

In [ ]:
# Finish model
updated_model.compile(optimizer='adam',loss='mse', metrics=['acc'])

In [ ]:
new_stopping_cb = keras.callbacks.EarlyStopping(patience=50, monitor='acc',
                                                restore_best_weights=True)

In [ ]:
updated_model.fit(imgs_for_input, imgs_for_output, epochs = 100, callbacks=[new_stopping_cb] )

In [ ]:
h = updated_model.history

plt.plot(h.history['acc'])
plt.title('Model accuracy')
plt.show()

plt.plot(h.history['loss'])
plt.title('Model Loss')
plt.show()

In [ ]:
output = updated_model.predict(imgs_for_input)

In [ ]:
plt.imshow(output[25])

In [ ]:
plt.imshow(imgs_for_output[25])

# Re-running updated CNN with Accuracy Stopping

In [ ]:
acc_stopping_cb = keras.callbacks.EarlyStopping(patience=500, monitor='acc',
                                                restore_best_weights=True)

In [ ]:
model.fit(imgs_for_input, imgs_for_output, epochs = 1000, callbacks=[acc_stopping_cb])

In [ ]:
h = model.history

plt.plot(h.history['acc'])
plt.title('Model accuracy')
plt.show()

plt.plot(h.history['loss'])
plt.title('Model Loss')
plt.show()

# Predicting Accuracy CNN Outputs

In [ ]:
output = model.predict(imgs_for_input)

In [ ]:
plt.imshow(output[25])

In [ ]:
plt.imshow(imgs_for_output[25])

# Data Augmentation and Feature Mapping

In [ ]:
imgs_for_input.shape

In [ ]:
imgs_for_input[0].shape

In [ ]:
# The following code loads two sample images, then it creates two filters
# and applies them to both images; then it displays one of the resulting
# feature maps

# from sklearn.datasets import load_sample_image
# Load sample images
# china = load_sample_image('china.jpg')/255
# flower = load_sample_image('flower.jpg')/255

image = imgs_for_input
batch_size, height, width, channels = imgs_for_input.shape

# Create two filters
filters = np.zeros(shape=(7,7, channels, 2), dtype=np.float32)
filters[:, 3, :, 0] = 1 # Vertical Line
filters[3, :, :, 1] = 1 # Horizontal Line

outputs = tf.nn.conv2d(image, filters, strides=1, padding='SAME')

plt.figure(figsize=(18,10))
plt.imshow(outputs[0, :, :, 1], cmap='gray') # Plot the first image's 2nd feature map
plt.show()

plt.figure(figsize=(18,10))
plt.imshow(outputs[0,:,:,0], cmap='gray')

# Pixel intensity is represented as a byte from 0 to 255, so we scale these features
# simply by dividing by 255. Then we create two 7x7 filters. We apply them both
# using the tf.nn.conv2d() function. We use zero padding with a stride of 2.
# tf.nn.conv2d() deserves more explanation: images is the input mini-batch of 
# 4 dimensions. Filters is the set of filters to apply (also a 4d tensor)/
# Strides is equal to 1, but it could also be a 1D array with four elements, 
# wherethe two central elements are the vertical and horizontal strides. The 
# beginning and trailing 1s could later be used to specify a batch stride (to skip
# some instances) and a chennel stride (to skip some of the previous layer's feature
# maps or channels). If padding is set to same, the convolutional layer uses zero padding
# if necessary. The output size is set to the numberof input neurons dividided by the
# stride, rounded up. Zeros are added as evenly around the inputs as needed. When
# strides = 1, the layer's outputs will have the same spatial dimensions (width
# and height) as its inputs, hence the name same. If set to 'valid' the conv layer
# does not use zero padding and may ignor some rows and colums at the bottom and right
# of the image, depending on the stride. This means every neuron's receptive field
# lies strictly within valid positions inside the input (it does not go out of bounds)

# Normalizing -- Another Method

In [ ]:
woman_img = array_to_img(imgs_for_output[25])

In [ ]:
woman_img

In [ ]:
imsave('tech_woman.jpg', imgs_for_output[25])

In [ ]:
image = imgs_output[25]

In [ ]:
plt.imshow(imgs_output[25])

In [ ]:
X = rgb2lab(1.0/255*image)[:,:,0]

In [ ]:
Y = rgb2lab(1.0/255*image)[:,:,1:]

In [ ]:
Y /=128

In [ ]:
X = X.reshape(1, 224, 224, 1)

In [ ]:
Y = Y.reshape(1, 224, 224, 2)

In [ ]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=1000)

In [ ]:
output = model.predict(X)

In [ ]:
output *=128

In [ ]:
# Output colorizations
cur = np.zeros((224,224, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

In [ ]:
plt.imshow(cur)

# ImageDataGenerator and Feature Mapping

In [ ]:
L_datagen = ImageDataGenerator(rescale=1./100, shear_range=0.2, zoom_range=0.2,
                               rotation_range=20, horizontal_flip=True,
                               preprocessing_function=preprocess_input)

AB_datagen = ImageDataGenerator(rescale=1./128, shear_range=0.2, zoom_range=0.2,
                                rotation_range=20, horizontal_flip=True,
                                preprocessing_function=preprocess_input)

RGB_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2,
                                 rotation_range=20, horizontal_flip=True,
                                 preprocessing_function=preprocess_input) 

In [ ]:
input_datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2,
                                   rotation_range=20, horizontal_flip=True)

output_datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2,
                                    rotation_range=20, horizontal_flip=True)

datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2,
                             rotation_range=20, horizontal_flip=True)

In [ ]:
imgs_input.dtype

In [ ]:
imgs_output.dtype

In [ ]:
grey_full = normalize_L(imgs_input)

In [ ]:
def normalize_RGB(data):
    rgb = []
    for img in np.nditer(data):
        img = img*(1.0/255)
        rgb.append(img)
    rgb = np.array(rgb, 'float')
    return rgb

In [ ]:
rgb_full = (imgs_output/128)

In [ ]:
aug_data = tf.keras.preprocessing.image.NumpyArrayIterator(imgs_input,imgs_output, datagen, batch_size=300)

In [ ]:
aug_data.dtype

In [ ]:
grey_four_rank = L_images.reshape(25000, 224, 224, 1)

In [ ]:
# greyscale_train = L_datagen.flow(grey_four_rank, batch_size=(300))

In [ ]:
# ab_train = AB_datagen.flow(AB_images, batch_size=(300))

# Training on Augmented Data

In [ ]:
# Building the neural network
datagen_model = Sequential()
datagen_model.add(InputLayer(input_shape=(224,224,3)))
datagen_model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
datagen_model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
datagen_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
datagen_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
datagen_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
datagen_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
datagen_model.add(UpSampling2D((2, 2)))
datagen_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
datagen_model.add(UpSampling2D((2, 2)))
datagen_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
datagen_model.add(UpSampling2D((2, 2)))
datagen_model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))

In [ ]:
datagen_model.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [ ]:
datagen_model.fit(aug_data, epochs=10)

In [ ]:
datagen_output = datagen_model.predict(imgs_input)

In [ ]:
plt.imshow(datagen_output[25])

# Best Model -- Normalizing AB images

In [ ]:
def normalize_L(data):
    for img in np.nditer(data):
        img = img*(1.0/255)
    return data

In [ ]:
L_images = L_images.astype('float')

In [ ]:
AB_images = AB_images.astype('float')

In [ ]:
def normalize_ab(data):
    for img in np.nditer(data):
        img = img*(1.0/128)
    return data

In [ ]:
y = normalize_ab(AB_images)

In [ ]:
train = L_images[25].reshape(1, 224, 224, 1)

In [ ]:
target = y[25].reshape(1, 224, 224, 2)

# CNN after Data Augmentation

In [ ]:
aug_model = Sequential()
aug_model.add(InputLayer(input_shape=(None, None, 1)))
aug_model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
aug_model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
aug_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
aug_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
aug_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
aug_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
aug_model.add(UpSampling2D((2, 2)))
aug_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
aug_model.add(UpSampling2D((2, 2)))
aug_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
aug_model.add(UpSampling2D((2, 2)))
aug_model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
# Finish model
aug_model.compile(optimizer='rmsprop',loss='mse', metrics=['acc'])

In [ ]:
aug_model.fit(train, target, batch_size=1, epochs=1000)

In [ ]:
aug_output = aug_model.predict(train)

In [ ]:
aug_output *= 128

In [ ]:
cur = np.zeros((224,224,3))
cur[:,:,0] = train[0][:,:,0]
cur[:,:,1:] = aug_output[0]

color_prediction = lab2rgb(cur)

In [ ]:
plt.imshow(color_prediction)

# Code Betwixt

In [ ]:
# Instead of manually creating the variables, use the keras.layers.Conv2D layer
conv = keras.layers.Conv2d(filters=32, kernel_size=3, strides=1,
                           padding='same', activation='relu')
# kernel_size specifies the filter size, so 3x3. As always, we can use
# cross validation to find the right hyperparameter values, but this is very
# time consuming. 

In [ ]:
# A single CNN to tackle the Fashion MNIST dataset
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation='relu', padding='same',
                        input_shape=[28,28,1]),
    keras.layers.MaxPooling2D(2),
# Page 461.

# The first layer uses 64 fairly larger filters (7x7) but no stride because the input
# images are not very large. Next we have a max pooling layer which uses a pool
# size of 2, so it divides each spatial dimension by a factor of 2. The number of
# filters increases from 64 to 128 and then to 256 as we move through the model.
# It is common practice to double the number of filters after each pooling layer:
# since a pooling layer divides each spatial dimension by a factor of 2, we can
# afford to increase our filters by the same factor without exploding the number of
# parameters. Next is a fully connected network composed of two hidden dense layers
# and a dense output layer. Note that we must flatten the inputs, since a dense network
# expects a 1D array of features for each instance. We also add two dropout layers
# with a dropout rate of 50% to reduce overfitting.
                                 
])

In [ ]:
# Build the ResNet-34 using a Sequential Model. Pg. 479

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64,7, stride=2, input_shape=[224,224,3],
                              padding='same', use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, acitvation='softmax'))

# Pretrained Models, ResNet-50 pg. 480  

# Recommendations and Future Goals